In [1]:
import numpy as np
import tensorflow as tf
import re
import time
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from sklearn.feature_extraction.text import CountVectorizer
from keras import layers , activations , models , preprocessing , utils

/home/bahbbc/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


## Dataset do cornell_movie_dialogs_corpus

In [2]:
lines = open('movie_lines.txt', encoding='utf-8', errors='ignore').read().split('\n')
conversations = open('movie_conversations.txt', encoding='utf-8', errors='ignore').read().split('\n')

In [3]:
lines[:10]

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?']

In [4]:
conversations[:10]

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']"]

In [5]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "can not", text)
    text = re.sub(r"[-()\"#/@;:<>{}+=~|.?,]", "", text)
    return text

## Limpa a sujeira do dataset original

In [6]:
id2line = {}
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [7]:
# Creating a list of all of the conversations
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))

# Getting separately the questions and the answers
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [8]:
# Cleaning the questions
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

# Cleaning the answers
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [9]:
questions[0]

'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.'

In [10]:
answers[0]

"Well, I thought we'd start with pronunciation, if that's okay with you."

In [11]:
for i in range(len(clean_answers)):
     clean_answers[i] = '<SOS> ' + clean_answers[i] + ' <EOS>'

In [12]:
clean_answers[:10]

['<SOS> well i thought we would start with pronunciation if that is okay with you <EOS>',
 '<SOS> not the hacking and gagging and spitting part  please <EOS>',
 "<SOS> okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 '<SOS> forget it <EOS>',
 '<SOS> cameron <EOS>',
 '<SOS> the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i can not date until she does <EOS>',
 '<SOS> seems like she could get a date easy enough <EOS>',
 '<SOS> unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 '<SOS> that is a shame <EOS>',
 '<SOS> let me see what i can do <EOS>']

## Pre processamento textual

In [13]:
# batch_size = 64  # Batch size for training.
# epochs = 100  # Number of epochs to train for.
# latent_dim = 256  # Latent dimensionality of the encoding space.
# num_samples = 10000  # Number of samples to train on.

In [14]:
batch_size = 32  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 64  # Latent dimensionality of the encoding space.
num_samples = 100  # Number of samples to train on.

In [15]:
vectorizer = CountVectorizer()

In [16]:
vectorizer.fit(clean_questions)
questions_words = vectorizer.vocabulary_.keys()

In [17]:
vectorizer.fit(clean_answers)
answers_words = vectorizer.vocabulary_.keys()

In [18]:
MAX_NB_WORDS = 100
MAX_SEQUENCE_LENGTH = 100

In [19]:
input_characters = sorted(list(questions_words))
target_characters = sorted(list(answers_words))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in clean_questions])
max_decoder_seq_length = max([len(txt) for txt in clean_answers])

print('Number of samples:', len(clean_questions))
print('Number of unique input words:', num_encoder_tokens)
print('Number of unique output words:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 221616
Number of unique input words: 52247
Number of unique output words: 53062
Max sequence length for inputs: 1856
Max sequence length for outputs: 2887


In [20]:
questions = clean_questions[:2000]

In [21]:
answers = clean_answers[:2000]

In [22]:
answers

['<SOS> well i thought we would start with pronunciation if that is okay with you <EOS>',
 '<SOS> not the hacking and gagging and spitting part  please <EOS>',
 "<SOS> okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 '<SOS> forget it <EOS>',
 '<SOS> cameron <EOS>',
 '<SOS> the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i can not date until she does <EOS>',
 '<SOS> seems like she could get a date easy enough <EOS>',
 '<SOS> unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 '<SOS> that is a shame <EOS>',
 '<SOS> let me see what i can do <EOS>',
 '<SOS> right  see  you are ready for the quiz <EOS>',
 "<SOS> i don't want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone <EOS>",
 "<

In [23]:
input_characters = sorted(list(questions))
target_characters = sorted(list(answers))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in questions])
max_decoder_seq_length = max([len(txt) for txt in answers])

print('Number of samples:', len(questions))
print('Number of unique input words:', num_encoder_tokens)
print('Number of unique output words:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 1000
Number of unique input words: 1000
Number of unique output words: 1000
Max sequence length for inputs: 496
Max sequence length for outputs: 655


In [24]:
# Keras 2.16 and tf 1.4.1

In [25]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, filters='')
tokenizer.fit_on_texts(questions)
enconder_sequences = tokenizer.texts_to_sequences(questions)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 1892 unique tokens.


In [26]:
tokenizer.fit_on_texts(answers)
decoder_sequences = tokenizer.texts_to_sequences(answers)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 2395 unique tokens.


In [27]:
word_dict = tokenizer.word_index

In [28]:
word_dict['<sos>']

2

In [29]:
decoder_target_data = list()
for token_seq in decoder_sequences:
    decoder_target_data.append(token_seq[1 : ]) 
    
padded_anwers = sequence.pad_sequences(decoder_target_data, maxlen=MAX_NB_WORDS)
onehot_answers = utils.to_categorical(padded_anwers, num_decoder_tokens)
decoder_target_data = np.array(onehot_answers)
print('Decoder target data shape -> {}'.format(decoder_target_data.shape))

Decoder target data shape -> (1000, 100, 1000)


In [30]:
encoder_input_data = sequence.pad_sequences(enconder_sequences, maxlen=MAX_NB_WORDS)
decoder_input_data = sequence.pad_sequences(decoder_sequences, maxlen=MAX_NB_WORDS)


print('Shape of encoder:', encoder_input_data.shape)
print('Shape of decoder:', decoder_input_data.shape)
print('Shape of target:', decoder_target_data.shape)

Shape of encoder: (1000, 100)
Shape of decoder: (1000, 100)
Shape of target: (1000, 100, 1000)


In [31]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
enconder_lstm, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(encoder_embedding)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
output = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], output)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
# Note that `decoder_target_data` needs to be one-hot encoded,
# rather than sequences of integers like `decoder_input_data`!
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     64000       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 64)     64000       input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LS

In [32]:
model.save('model.h5') 

/home/bahbbc/anaconda3/lib/python3.6/site-packages/keras/engine/topology.py:2379: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 64) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 64) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [33]:
max_question_len = encoder_input_data.shape[1]
max_answer_len = decoder_input_data.shape[1]

In [34]:
def make_inference_models():    
    encoder_model = keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = Input(shape=(latent_dim ,))
    decoder_state_input_c = keras.layers.Input(shape=(latent_dim ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [35]:
def str_to_tokens(sentence : str ):
    words = sentence.lower().split()
    tokens_list = list()
    for word in words:
        tokens_list.append( word_dict[ word ] ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=max_question_len , padding='post')

In [37]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = word_dict['<sos>']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in word_dict.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == '<eos>' or len(decoded_translation.split()) > max_answer_len:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print(decoded_translation)

Enter question : hi


FailedPreconditionError: Attempting to use uninitialized value dense_1/kernel
	 [[Node: dense_1/kernel/read = Identity[T=DT_FLOAT, _class=["loc:@dense_1/kernel"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](dense_1/kernel)]]

Caused by op 'dense_1/kernel/read', defined at:
  File "/home/bahbbc/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/bahbbc/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/bahbbc/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/bahbbc/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/bahbbc/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/home/bahbbc/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.p

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence